In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1460071,2021-07-13,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1460072,2021-07-14,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1460073,2021-07-15,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1460074,2021-07-16,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
38491,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
38493,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
38495,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
38497,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
38499,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
893579,2021-07-13,Arkansas,Arkansas,5001,2302,40.00,5000,Arkansas,AR,Arkansas,State,3017804
893581,2021-07-14,Arkansas,Arkansas,5001,2320,40.00,5000,Arkansas,AR,Arkansas,State,3017804
893583,2021-07-15,Arkansas,Arkansas,5001,2336,40.00,5000,Arkansas,AR,Arkansas,State,3017804
893585,2021-07-16,Arkansas,Arkansas,5001,2358,40.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1460071,2021-07-13,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1460072,2021-07-14,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1460073,2021-07-15,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1460074,2021-07-16,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-07-17') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
543,2021-07-17,Snohomish,Washington,41194,613.00,53061,WA,County,822083,5010.93,74.57
1084,2021-07-17,Cook,Illinois,558928,11094.00,17031,IL,County,5150233,10852.48,215.41
1624,2021-07-17,Orange,California,275358,5136.00,6059,CA,County,3175692,8670.80,161.73
2163,2021-07-17,Maricopa,Arizona,568357,10388.00,4013,AZ,County,4485414,12671.23,231.60
2702,2021-07-17,Los Angeles,California,1266908,24582.00,6037,CA,County,10039107,12619.73,244.86
...,...,...,...,...,...,...,...,...,...,...,...
1459087,2021-07-17,Wheeler,Oregon,35,1.00,41069,OR,County,1332,2627.63,75.08
1459365,2021-07-17,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1459612,2021-07-17,Esmeralda,Nevada,40,0.00,32009,NV,County,873,4581.90,0.00
1459855,2021-07-17,Loving,Texas,3,0.00,48301,TX,County,169,1775.15,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
543,2021-07-17,Snohomish,Washington,41194,613.00,53061,WA,County,822083,5010.93,74.57,74.57,5010.93
1084,2021-07-17,Cook,Illinois,558928,11094.00,17031,IL,County,5150233,10852.48,215.41,215.41,10852.48
1624,2021-07-17,Orange,California,275358,5136.00,6059,CA,County,3175692,8670.80,161.73,161.73,8670.80
2163,2021-07-17,Maricopa,Arizona,568357,10388.00,4013,AZ,County,4485414,12671.23,231.60,231.60,12671.23
2702,2021-07-17,Los Angeles,California,1266908,24582.00,6037,CA,County,10039107,12619.73,244.86,244.86,12619.73
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1459087,2021-07-17,Wheeler,Oregon,35,1.00,41069,OR,County,1332,2627.63,75.08,75.08,2627.63
1459365,2021-07-17,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1459612,2021-07-17,Esmeralda,Nevada,40,0.00,32009,NV,County,873,4581.90,0.00,0.00,4581.90
1459855,2021-07-17,Loving,Texas,3,0.00,48301,TX,County,169,1775.15,0.00,0.00,1775.15


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
934520,2021-07-17,Hale,Alabama,2295,78.00,1065,AL,County,14651,15664.46,532.39,532.39,15664.46,1
1017565,2021-07-17,Clarke,Alabama,3582,62.00,1025,AL,County,23622,15163.83,262.47,262.47,15163.83,2
730581,2021-07-17,Lowndes,Alabama,1434,54.00,1085,AL,County,9726,14743.99,555.21,555.21,14743.99,3
647547,2021-07-17,Etowah,Alabama,14414,368.00,1055,AL,County,102268,14094.34,359.84,359.84,14094.34,4
543096,2021-07-17,Franklin,Alabama,4359,81.00,1059,AL,County,31362,13898.99,258.27,258.27,13898.99,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
892159,2021-07-17,Sublette,Wyoming,819,7.00,56035,WY,County,9831,8330.79,71.20,71.20,8330.79,19
891682,2021-07-17,Converse,Wyoming,1065,18.00,56009,WY,County,13822,7705.11,130.23,130.23,7705.11,20
1109607,2021-07-17,Lincoln,Wyoming,1461,14.00,56023,WY,County,19830,7367.62,70.60,70.60,7367.62,21
1235783,2021-07-17,Niobrara,Wyoming,163,3.00,56027,WY,County,2356,6918.51,127.33,127.33,6918.51,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
730581,2021-07-17,Lowndes,Alabama,1434,54.00,1085,AL,County,9726,14743.99,555.21,555.21,14743.99,3,1
934520,2021-07-17,Hale,Alabama,2295,78.00,1065,AL,County,14651,15664.46,532.39,532.39,15664.46,1,2
310012,2021-07-17,Walker,Alabama,7488,286.00,1127,AL,County,63521,11788.23,450.24,450.24,11788.23,29,3
645627,2021-07-17,Clay,Alabama,1610,59.00,1027,AL,County,13235,12164.71,445.79,445.79,12164.71,21,4
647067,2021-07-17,Dallas,Alabama,3657,161.00,1047,AL,County,37196,9831.70,432.84,432.84,9831.70,55,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1109607,2021-07-17,Lincoln,Wyoming,1461,14.00,56023,WY,County,19830,7367.62,70.60,70.60,7367.62,21,19
786145,2021-07-17,Hot Springs,Wyoming,382,3.00,56017,WY,County,4413,8656.24,67.98,67.98,8656.24,17,20
1081818,2021-07-17,Uinta,Wyoming,2335,13.00,56041,WY,County,20226,11544.55,64.27,64.27,11544.55,5,21
308554,2021-07-17,Teton,Wyoming,3835,11.00,56039,WY,County,23464,16344.19,46.88,46.88,16344.19,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,74.57,5010.93,21,25
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,74.57,5010.93,21,25
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,74.57,5010.93,21,25
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,74.57,5010.93,21,25
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,74.57,5010.93,21,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456669,2021-07-13,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1456670,2021-07-14,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1456671,2021-07-15,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1456672,2021-07-16,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
930644,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15664.46,2,1,1.00
930645,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15664.46,2,1,0.00
930646,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15664.46,2,1,0.00
930647,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,532.39,15664.46,2,1,0.00
930648,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,532.39,15664.46,2,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128291,2021-07-13,Crook,Wyoming,465,13.00,56011,WY,County,7584,6131.33,171.41,171.41,6157.70,6,23,0.00
1128292,2021-07-14,Crook,Wyoming,466,13.00,56011,WY,County,7584,6144.51,171.41,171.41,6157.70,6,23,1.00
1128293,2021-07-15,Crook,Wyoming,467,13.00,56011,WY,County,7584,6157.70,171.41,171.41,6157.70,6,23,1.00
1128294,2021-07-16,Crook,Wyoming,467,13.00,56011,WY,County,7584,6157.70,171.41,171.41,6157.70,6,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
727655,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14743.99,1,3,1.00,0.00
727656,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14743.99,1,3,0.00,0.00
727657,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14743.99,1,3,0.00,0.00
727658,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14743.99,1,3,0.00,0.00
727659,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,555.21,14743.99,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725255,2021-07-13,Albany,Wyoming,4680,17.00,56001,WY,County,38880,12037.04,43.72,43.72,12080.76,23,3,1.00,0.00
725256,2021-07-14,Albany,Wyoming,4687,17.00,56001,WY,County,38880,12055.04,43.72,43.72,12080.76,23,3,7.00,0.00
725257,2021-07-15,Albany,Wyoming,4695,17.00,56001,WY,County,38880,12075.62,43.72,43.72,12080.76,23,3,8.00,0.00
725258,2021-07-16,Albany,Wyoming,4697,17.00,56001,WY,County,38880,12080.76,43.72,43.72,12080.76,23,3,2.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-07-17') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
370244,2021-07-17,Imperial,California,28840,741.00,6025,CA,County,181215,15914.80,408.91,408.91,15914.80,1,2,0.00,0.00
2702,2021-07-17,Los Angeles,California,1266908,24582.00,6037,CA,County,10039107,12619.73,244.86,244.86,12619.73,2,5,1777.00,0.00
184050,2021-07-17,San Bernardino,California,302238,5218.00,6071,CA,County,2180085,13863.59,239.35,239.35,13863.59,3,4,0.00,0.00
649419,2021-07-17,Inyo,California,1444,38.00,6027,CA,County,18039,8004.88,210.65,210.65,8004.88,4,25,0.00,0.00
82153,2021-07-17,Stanislaus,California,64381,1082.00,6099,CA,County,550660,11691.61,196.49,196.49,11691.61,5,9,0.00,0.00
65325,2021-07-17,San Joaquin,California,75500,1446.00,6077,CA,County,762148,9906.21,189.73,189.73,9906.21,6,17,0.00,0.00
37992,2021-07-17,Riverside,California,302069,4647.00,6065,CA,County,2470546,12226.81,188.10,188.10,12226.81,7,7,0.00,0.00
101910,2021-07-17,Tulare,California,49964,854.00,6107,CA,County,466195,10717.40,183.19,183.19,10717.40,8,11,0.00,0.00
588349,2021-07-17,Merced,California,32621,483.00,6047,CA,County,277680,11747.70,173.94,173.94,11747.70,9,8,0.00,0.00
36996,2021-07-17,Fresno,California,103055,1730.00,6019,CA,County,999101,10314.77,173.16,173.16,10314.77,10,14,0.00,0.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1379977,2021-07-17,Lassen,California,5858,26.00,6035,CA,County,30573,19160.70,85.04,85.04,19160.70,35,1,0.00,0.00
370244,2021-07-17,Imperial,California,28840,741.00,6025,CA,County,181215,15914.80,408.91,408.91,15914.80,1,2,0.00,0.00
788000,2021-07-17,Kings,California,23255,248.00,6031,CA,County,152940,15205.31,162.16,162.16,15205.31,11,3,15.00,0.00
184050,2021-07-17,San Bernardino,California,302238,5218.00,6071,CA,County,2180085,13863.59,239.35,239.35,13863.59,3,4,0.00,0.00
2702,2021-07-17,Los Angeles,California,1266908,24582.00,6037,CA,County,10039107,12619.73,244.86,244.86,12619.73,2,5,1777.00,0.00
231525,2021-07-17,Kern,California,111687,1414.00,6029,CA,County,900202,12406.88,157.08,157.08,12406.88,14,6,0.00,0.00
37992,2021-07-17,Riverside,California,302069,4647.00,6065,CA,County,2470546,12226.81,188.10,188.10,12226.81,7,7,0.00,0.00
588349,2021-07-17,Merced,California,32621,483.00,6047,CA,County,277680,11747.70,173.94,173.94,11747.70,9,8,0.00,0.00
82153,2021-07-17,Stanislaus,California,64381,1082.00,6099,CA,County,550660,11691.61,196.49,196.49,11691.61,5,9,0.00,0.00
787522,2021-07-17,Colusa,California,2322,18.00,6011,CA,County,21547,10776.44,83.54,83.54,10776.44,36,10,0.00,0.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
370244,2021-07-17,Imperial,California,28840,741.00,6025,CA,County,181215,15914.80,408.91,408.91,15914.80,1,2,0.00,0.00
2702,2021-07-17,Los Angeles,California,1266908,24582.00,6037,CA,County,10039107,12619.73,244.86,244.86,12619.73,2,5,1777.00,0.00
184050,2021-07-17,San Bernardino,California,302238,5218.00,6071,CA,County,2180085,13863.59,239.35,239.35,13863.59,3,4,0.00,0.00
649419,2021-07-17,Inyo,California,1444,38.00,6027,CA,County,18039,8004.88,210.65,210.65,8004.88,4,25,0.00,0.00
82153,2021-07-17,Stanislaus,California,64381,1082.00,6099,CA,County,550660,11691.61,196.49,196.49,11691.61,5,9,0.00,0.00
65325,2021-07-17,San Joaquin,California,75500,1446.00,6077,CA,County,762148,9906.21,189.73,189.73,9906.21,6,17,0.00,0.00
37992,2021-07-17,Riverside,California,302069,4647.00,6065,CA,County,2470546,12226.81,188.10,188.10,12226.81,7,7,0.00,0.00
101910,2021-07-17,Tulare,California,49964,854.00,6107,CA,County,466195,10717.40,183.19,183.19,10717.40,8,11,0.00,0.00
588349,2021-07-17,Merced,California,32621,483.00,6047,CA,County,277680,11747.70,173.94,173.94,11747.70,9,8,0.00,0.00
36996,2021-07-17,Fresno,California,103055,1730.00,6019,CA,County,999101,10314.77,173.16,173.16,10314.77,10,14,0.00,0.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...
4847,07/13/21,Colusa,2308,18.00,10711.47,83.54,36,10,10.00,0.00
4848,07/14/21,Colusa,2308,18.00,10711.47,83.54,36,10,0.00,0.00
4849,07/15/21,Colusa,2308,18.00,10711.47,83.54,36,10,0.00,0.00
4850,07/16/21,Colusa,2322,18.00,10776.44,83.54,36,10,14.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...
4948,07/13/21,Fresno,102853,1728.00,10294.55,172.96,10,14,40.00,0.00
4949,07/14/21,Fresno,102909,1727.00,10300.16,172.86,10,14,56.00,-1.00
4950,07/15/21,Fresno,102966,1728.00,10305.86,172.96,10,14,57.00,1.00
4951,07/16/21,Fresno,103055,1730.00,10314.77,173.16,10,14,89.00,2.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)